# Ejercicio Regression Tree
Son datos de temperaturas tomados de la ciudad de Seattle. El objetivo es intentar predecir lo mejor posible la máxima temperatura (columna `actual`) que alcanzaremos al día siguiente. Para ello contaremos con temperaturas de los dos días anteriores, históricos de máximas temperaturas y un amigo bastante chapas que te da su predicción del tiempo cada día.

**Resumen de datos**:
* year: 2016 for all data points
* month: number for month of the year
* day: number for day of the year
* week: day of the week as a character string
* temp_2: max temperature 2 days prior
* temp_1: max temperature 1 day prior
* average: historical average max temperature
* actual: max temperature measurement
* friend: your friend’s prediction, a random number between 20 below the average and 20 above the average


### 1. Importa el csv "temps.csv"

### 2. Realiza un análisis exploratorio junto con un Feature Engineering

### 3. Selecciona variables y divide en train y test

### 4. Entrena el modelo


### 5. Evaluación de tu modelo

Calcula el MAE, MAPE, MSE y RMSE.

### 6. Representa el árbol de decisión

### 7. Modifica el max_depth a 3 y vuelve a entrenarlo

### 8. Vuelve a evaluar tu modelo

### 9. Vuelve a representar su árbol

### 10. Utiliza un grid search con cross validation para hiperparametrizar tu modelo

In [ ]:
model = DecisionTreeRegressor()

parameters = {
    "max_depth" : [2,3,4,5]
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error")

In [ ]:
#best_model = dtr_gs.best_estimator_

In [40]:
y_pred = dtr_gs.best_estimator_.predict(X_test)

print("MAE", mean_absolute_error(y_test, y_pred))
print("MAPE", mean_absolute_percentage_error(y_test, y_pred))
print("MSE", mean_squared_error(y_test, y_pred))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred)))

MAE 4.050098189195934
MAPE 0.06409047399308641
MSE 28.795479510450583
RMSE 5.36614195772443


In [ ]:
dtr_gs.best_params_

### 11. Obten el `feature_importances` de cada variable en el último modelo

### 12. Construye un nuevo modelo `DecisionTreeClassifier` que te clasifique cuando en un día va a hacer una temperatura mayor a 15ºC. ¡Deberá tratarse como un problema de **clasificación**! Utiliza un grid search con cross validation y evalúa el mejor modelo.